In [276]:
import numpy as np
import pandas as pd
import networkx as nx
import time
import timeit
from datetime import datetime,timedelta
import http.client
import requests
import json
import yfinance as yf
#%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from heapq import *
# import planarity
from sklearn.linear_model import LinearRegression

In [280]:
#!pip install finta
from finta import TA
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [2]:
changes_df = pd.read_excel('S&P500 changes.xlsx',index_col='date')
changes_df.index = pd.to_datetime(changes_df.index)
changes_df = changes_df.loc['2017-04-01':]
changes_df.tail()

<ipython-input-2-0e917c9ae7a8>:3: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  changes_df = changes_df.loc['2017-04-01':]


,Added Ticker,Added Security,Removed Ticker,Removed Security,Reason
date,,,,,
2017-06-19,ANSS,Ansys,R,Ryder System,Market capitalization changes.[74]
2017-06-19,RE,Everest Re,MJN,Mead Johnson,Reckitt Benckiser acquired Mead Johnson Nutrit...
2017-06-02,INFO,IHS Markit,TGNA,Tegna,TGNA spins off Cars.com[76]
2017-04-05,IT,Gartner,DNB,Dun & Bradstreet,IT acquiring CEB[77]
2017-04-04,DXC,DXC Technology,SWN,Southwestern Energy,"HPE spins off Everett, merged with CSC to form..."


In [3]:
cur_components_df = pd.read_excel('S&P500 current components.xlsx')
cur_components_df.head()

,Symbol,Security,SEC filings,GICS聽Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,A,Agilent Technologies,reports,Health Care,Health Care Equipment,"Santa Clara, California",36682,1090872,1999
1,AAL,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",42086,6201,1934
2,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",42194,1158449,1932
3,AAPL,Apple Inc.,reports,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",30285,320193,1977
4,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",41274,1551152,2013 (1888)


In [4]:
# generate S&P500 components from now to historical periods.
# initialize
components_mark_df = pd.DataFrame()
prev_date = 0
stocks = list(cur_components_df['Symbol'])
# generate historical components based on changes
for date in changes_df.index[:-1]:
    if date != prev_date:
        components_mark_df.loc[date, stocks] = 1
        prev_date = date
        removed = changes_df.loc[date, 'Removed Ticker']
        added = changes_df.loc[date, 'Added Ticker']
        
        if type(removed) == str:
            stocks.append(removed)            
        elif isinstance(removed, pd.Series):
            for ele in list(removed.values):
                if type(ele) == str: stocks.append(ele)
                    
                    
        if type(added) == str:
            stocks.remove(added)
        elif isinstance(added, pd.Series):
            for ele in list(added.values):
                if type(ele) == str: stocks.remove(ele)
                    
components_mark_df = components_mark_df.sort_index()
components_mark_df.head()
#components mark df 是所有的股票在变动日那天是否存在于SP500的boolean dataframe, nan为不在, 1 为存在

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,RIG,BBBY,MUR,MNK,RAI,YHOO,TDC,R,MJN,TGNA
2017-04-05,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2017-06-02,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-06-19,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2017-07-26,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-08,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [278]:
startdate = '2021-06-01'
# startdate = datetime.strptime(startdate, '%Y-%m-%d')-timedelta(days=60)
enddate = '2022-06-28'
tickers = list(components_mark_df.columns)
components_price_df = yf.download(tickers, startdate, enddate,)['Adj Close']

components_price_df.index = pd.to_datetime(components_price_df.index)
# components_price_df.to_csv('components_price_new.csv')

[*********************100%***********************]  604 of 604 completed

51 Failed downloads:
- PBCT: No data found, symbol may be delisted
- GGP: No data found for this date range, symbol may be delisted
- VAR: No data found, symbol may be delisted
- MXIM: No data found, symbol may be delisted
- ESRX: No data found for this date range, symbol may be delisted
- YHOO: No data found for this date range, symbol may be delisted
- AET: No data found for this date range, symbol may be delisted
- SCG: No data found for this date range, symbol may be delisted
- DWDP: No data found, symbol may be delisted
- DPS: No data found for this date range, symbol may be delisted
- COL: No data found for this date range, symbol may be delisted
- KSU: No data found, symbol may be delisted
- ALXN: No data found, symbol may be delisted
- ANTM: No data found, symbol may be delisted
- LVLT: No data found for this date range, symbol may be delisted
- ANDV: No data found for this date range, symbol may be delis

In [43]:
#components_price_df = pd.read_csv('components_price_new.csv',index = 'Date')

In [322]:
components_price_df.index = pd.to_datetime(components_price_df.index)

for ticker in components_price_df.columns:
    # delete stock whose information can not be retrived
    if components_price_df[ticker].isnull().values.all():
        components_price_df = components_price_df.drop(columns=[ticker])
        #print(tick)
        continue
    # process abnormal data
    for d in components_price_df.index:
        if type(components_price_df.loc[d,ticker]) == str:
            components_price_df.loc[d,ticker]=np.nan
        else:
            if components_price_df.loc[d,ticker]<0 or components_price_df.loc[d,ticker]>9999:
                components_price_df.loc[d,ticker]=np.nan
# fillna with previous price
components_price_df = components_price_df.fillna(method="ffill")
components_price_df = components_price_df.fillna(method="bfill")

# Generating Features, use ticker with yfinance, read close/volume/high/low and use package to generate features.
# finta: https://github.com/peerchemist/finta
# need period, return null when valid data is less than 80% of the whole period
# ticker:'AMZN', start:'2019-01-01'
from finta import TA
def feature_generate(ticker,start,end):
    tk = yf.Ticker(ticker)
    ohlc = pd.DataFrame()
    ohlc['open'] = tk.history(start=start, end=end)['Open']
    ohlc["high"] = tk.history(start=start, end=end)['High']
    ohlc["low"] = tk.history(start=start, end=end)['Low']
    ohlc["close"] = tk.history(start=start, end=end)['Close']
    ohlc["volume"] = tk.history(start=start, end=end)['Volume']
    
    sta = pd.DataFrame()
    sta['return'] = (ohlc["close"] - ohlc["close"].shift())/ohlc["close"].shift()
    sta['open'] = ohlc['open']/ohlc["close"]
    sta["high"] = ohlc['high']/ohlc["close"]
    sta["low"] = ohlc['low']/ohlc["close"]
    sta["volume"] = (ohlc["volume"] - ohlc["volume"].shift())/ohlc["volume"].shift()
    
    df = pd.DataFrame()
    df['SMA']= TA.SMA(ohlc)
    df['SMM'] = TA.SMM(ohlc)
    df['ER'] = TA.ER(ohlc)
    df['MACD'] = TA.VW_MACD(ohlc).iloc[:,0]
    df['signal'] = TA.VW_MACD(ohlc).iloc[:,1]
    df['MOM'] = TA.MOM(ohlc)
    df['ROC'] = TA.ROC(ohlc)
    df['RSI'] = TA.RSI(ohlc)
    df['TR']= TA.TR(ohlc)
    df['SAR'] = TA.SAR(ohlc)
    df['BBANDS'] = TA.BBANDS(ohlc).iloc[:,1]
    df['BBWIDTH'] = TA.BBWIDTH(ohlc)
    df['KC_UPPER'] = TA.KC(ohlc).iloc[:,0]
    df['KC_LOWER'] = TA.KC(ohlc).iloc[:,1]
    df['DO'] = TA.DO(ohlc).iloc[:,1]
    df['DMI'] = TA.DMI(ohlc).iloc[:,1]
#     df['PIVOT'] = TA.PIVOT(ohlc).iloc[:,0]
#     df['WILLIAMS']= TA.WILLIAMS(ohlc)
#     df['UO'] = TA.UO(ohlc)
#     df['AO'] = TA.AO(ohlc)
#     df['MI'] = TA.MI(ohlc)
#     df['Vlm'] = TA.VORTEX(ohlc).iloc[:,0]
#     df['Vlp'] = TA.VORTEX(ohlc).iloc[:,1]
#     df['KST'] = TA.KST(ohlc).iloc[:,0]
#     df['TP'] = TA.TP(ohlc)
#     df['ADL'] = TA.ADL(ohlc)      
#     df['CHAIKIN']= TA.CHAIKIN(ohlc)
#     df['MFI'] = TA.MFI(ohlc)
# #     df['OBV'] = TA.OBV(ohlc)
#     df['EFI'] = TA.EFI(ohlc)
#     df['Bull'] = TA.EBBP(ohlc).iloc[:,0]
#     df['Bear'] = TA.EBBP(ohlc).iloc[:,1]
#     df['EMV'] = TA.EMV(ohlc)
#     df['CCI'] = TA.CCI(ohlc)
#     df['COPP']= TA.COPP(ohlc)
#     df['Buy'] = TA.BASP(ohlc).iloc[:,0]
#     df['Sell'] = TA.BASP(ohlc).iloc[:,1]
#     df['Short'] = TA.CHANDELIER(ohlc).iloc[:,0]
#     df['Long'] = TA.CHANDELIER(ohlc).iloc[:,1]
#     df['FISH'] = TA.FISH(ohlc)
#     df['VPT'] = TA.VPT(ohlc)
#     df['MSD'] = TA.MSD(ohlc)
#     df['STC'] = TA.STC(ohlc)

    final = sta.join(df)
    final.columns = pd.MultiIndex.from_product([[ticker], final.columns])
    return final

In [323]:
# input stocks, start, end,
# output merged multiindex dataframe with stocks and corresponding feature values
def merge_df(stocks,start,end):
    read_start = datetime.strptime(start, '%Y-%m-%d')-timedelta(days=60)
    test_df = feature_generate(stocks[0],read_start,end)
    for ticker in stocks[1:]:
        test_df = test_df.join(feature_generate(ticker,read_start,end))
    df = test_df.loc[start:end,:]
#     time = df.index.to_list()
    
    return df

# input stock price dataframe (will inplace price with pca values)
# output dataframe of stock pca dataframe
def pca_df(components_df, q, merged_df):
    pca = PCA(n_components=q)
#     count = 0
    for date in merged_df.index:
        df = merged_df.loc[[date]]
        df = df.reset_index().drop('Date', axis=1, level=0).T.reset_index()
        df.columns = ['ticker', 'type', 'value']
        df = df.pivot_table(index='ticker', columns='type', values='value')
        df = df.apply(lambda x: (x-np.mean(x))/np.std(x),axis=0)
    #   count += 1
    #     if sum(df.isnull().any()) > 0:
    #         nans.append(date)
    #     print(count)
        pca.fit_transform(df)
        weight = pca.components_[:1,:]
        pc = df.dot(weight.T)
        pc.apply(lambda x: (x-np.mean(x))/np.std(x),axis=0)
        for ticker in pc.index:
            components_df.loc[date][ticker] = pc.loc[ticker, 0]
    return components_df

In [355]:
valid_tickers = components_price_df.columns.values
merged_df = merge_df(valid_tickers[:30], '2022-06-01', '2022-07-01')

In [356]:
components_df = components_price_df.copy()
test_pca = pca_df(components_df.loc['2022-06-01':'2022-06-28'].iloc[:,:30], 1, merged_df.loc['2022-06-01':'2022-06-27'])

In [357]:
test_pca

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,AMAT,AMCR
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-01,-0.056663,-2.893451,2.464169,0.113395,0.300426,1.125104,4.626866,-1.016110,3.900112,7.333704,...,0.408149,0.399788,-1.613310,3.474881,6.626163,-1.662998,-0.851967,-1.482561,-0.706387,-4.642778
2022-06-02,0.786278,-3.712447,0.359213,0.140620,-1.271063,-1.780671,5.164249,-0.536889,4.504740,9.540763,...,-0.392900,0.571760,-1.381955,4.811001,6.035397,-2.964155,-1.189637,-1.255783,-0.618634,-3.242259
2022-06-03,-0.409636,-4.237154,0.688431,0.331910,-0.623569,-0.481897,4.191142,-0.791250,4.944305,9.230732,...,0.469773,0.456094,-1.235096,3.327887,4.177781,-3.302117,-0.271851,-1.347061,-0.703529,-3.483330
2022-06-06,-0.275899,-3.978723,0.979536,0.026525,-0.804390,-0.751468,4.582021,-0.744860,5.419171,8.895458,...,-0.118817,0.503928,-1.275706,2.997039,4.954875,-2.641702,-0.411580,-1.417716,-0.348780,-3.791292
2022-06-07,-0.063672,-3.889317,1.083810,0.196292,-0.546577,-0.617575,4.616498,-1.047078,5.087805,9.104498,...,0.590312,0.192324,-1.520611,3.679031,3.441196,-2.806896,-0.531513,-1.095980,-0.676868,-3.719241
2022-06-08,-0.568502,-4.534610,1.041389,0.295560,-0.389192,-1.111254,4.433732,-1.142215,5.417585,9.368544,...,0.425285,0.013379,-1.689457,4.456650,2.903384,-2.715488,-0.247845,-1.474925,-0.526459,-2.968008
2022-06-09,-0.244698,-4.332430,1.318450,-0.349345,-0.753514,-1.051632,3.402593,-0.909510,5.272485,10.597293,...,0.439676,-0.028628,-0.929076,3.959273,2.219642,-3.109514,-0.487425,-1.008715,-1.206592,-2.403179
2022-06-10,-0.287501,-3.425719,1.366058,0.172105,-0.585126,-0.542790,3.538958,-1.009231,4.579893,9.973813,...,0.532944,0.151091,-1.431073,3.831232,3.959689,-2.425251,-0.614398,-0.953083,-0.232298,-3.597211
2022-06-13,0.377310,-5.291579,1.356857,0.306229,0.055444,0.542223,4.352403,-0.821290,4.923799,8.518559,...,1.567573,1.183190,-2.188818,3.817426,4.957660,-3.479035,-0.148589,-0.472597,-0.982902,-3.426291


In [358]:
def PMFG(G):
    # sort edges weight in descending order
    h = []
    for u,v,d in G.edges(data=True):
        heappush(h,(d['weight'],u,v))
    heapsort = [heappop(h) for i in range(len(h))]
    heapsort.reverse()
    
    # PMFG algorithm
    res = nx.Graph()
    for (w,u,v) in heapsort:
        res.add_edge(u,v,weight=w)
        if not planarity.is_planar(res):
            res.remove_edge(u,v)
            
        if res.number_of_edges() == 3*(G.number_of_nodes()-2):
            break   

    return res
def PCA_distance(PCA_res):
    """To construct a 'matrix' representing 'weights' between points
    time-T, vector-q, number of stocks-N
    N * (q*T) matrix
    norm: distance norm 
    Output: N*N matrix"""
    N, q, T = PCA_res.shape
    res = np.zeros((N,N))
    for i in range(1,N):
        for j in range(i):
            Diff = PCA_res[i] - PCA_res[j]
            distance = sc.linalg.norm(Diff, ord='fro')
            res[i][j] = 1/distance   # larger distance means weaker correlation
    res = res + res.T  # symmetric matrix with diagonals vanish
    return res

In [359]:
relation_matrix = PCA_distance(test_pca)
G1 = nx.from_pandas_adjacency(relation_matrix)    # weights are 1/distance

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
G1_PMFG = PMFG(G1)
nx.draw(G1_PMFG, with_labels=False, node_size=25, node_color="#e1575c", edge_color='#363847', \
        pos=nx.kamada_kawai_layout(G1_PMFG))
p1_SS = dict_from_weights(relation_matrix)   # between [0,1]
G1_SS = SS(G1, p1_SS, 5, 120)

In [ ]:
G1_SS_p = WTA_proportion(G1_SS, 0, 0.1)  # the 0 doesn't matter, only 0.1 means to select top 10% edges
nx.draw(G1_SS_p, with_labels=False, node_size=25, node_color="#e1575c", edge_color='#363847', \
        pos=nx.kamada_kawai_layout(G1_SS_p))

In [ ]:
G1_SS_w = WTA_weight(G1_SS, 0.7, 1) # the 1 doesn't matter, only 0.7 means to select weights over 0.7
nx.draw(G1_SS_w, with_labels=False, node_size=25, node_color="#e1575c", edge_color='#363847', \
        pos=nx.kamada_kawai_layout(G1_SS_w))